#### Smali & Manifest File Analysis

In [1]:
from os import walk
import pandas as pd
import os
from tqdm import tqdm

In [2]:
#   TODO Add loading bar to this lengthy function
def decompileAll():

    # 1: Traverse all APKs in the folder:
    dir = 'C:\\Users\\Vin\\Desktop\\python-ransomware-analyzer\\benign'

    for filename in tqdm(os.listdir(dir)):
        if filename.endswith('.apk'):
            #print("decompiling: "+ filename)
            
            #set output directory (removing .apk from the end)
            foldername = 'out_' + filename[:-4]
            var1 = r'C:\Users\Vin\Desktop\python-ransomware-analyzer\decompiled_benign\\' + foldername
        
            #set input apk
            var2 = r'C:\Users\Vin\Desktop\python-ransomware-analyzer\benign\\' + filename
            
            #convert python vars to batch vars
            os.putenv("VAR1", var1)
            os.putenv("VAR2", var2)

            #call the run.bat for the apktool
            os.system("run.bat")
    print("...Done!")

In [3]:
#   Create empty lists
methods_to_find = []
perms_to_find = []

#   Read file using pandas
df = pd.read_csv('functions.csv', engine='python')
methods_to_find = df.functions

df = pd.read_csv('perms.csv', engine='python')
perms_to_find   = df.perms

In [4]:
#   Create class object to be used in the lists.
#   Object will contain the name of the feature and how many times it's been found
class obj:
    def __init__(self, name):
        self.name = name
        self.count = 0
    def print(self):
        print(self.name + ": " + str(self.count))

In [5]:
#   Initialize empty lists
myList = []
fakeList = []

#   Fill lists with permissions and functions. 
#   The "i" refers to a term in the list, not index.
for i in perms_to_find:
    myList.append(obj(i))
    fakeList.append(i)

for i in methods_to_find:
    myList.append(obj(i))
    fakeList.append(i)

#   Append the last item (name)
myList.append('')
fakeList.append('Name')

#   Self print function for list
def printList(list):
    for i in range(len(list)):
        try:
            list[i].print()
        except AttributeError:
            print(list[i])

In [6]:
#   Create empty dataframe
df = pd.DataFrame(columns=fakeList)

#   This clears the CSV if it exists
df.to_csv('benign_csv.csv', mode='w', header=True)

In [7]:
#   The analysis function
def findFunctionsInDir(direct, list=0, verbose = True, filename = ''):

    #for each directory, and files
    for dirpath, dirnames, files in walk(direct):
        
        #fill the name field in the list
        myList[fakeList.index('Name')] = filename

        #skip the res folder, too many subfolders, slows down.
        if('res' in dirpath): continue

        if(verbose == True): print(f'Found directory: {dirpath}')
        for file_name in files:
            
            #if it is a manifest file
            if('.xml' in file_name and 'anifest' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                #I had to use latin encoding here or it would simply not work.
                with open(dirpath + "\\" + file_name, encoding="latin-1") as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in perms_to_find:
                            if(i in line and ('<uses-permission android:name=' in line)
                                         or  ('<action android:name="android.app.action') in line):
                                
                                #Set permission found to 1
                                myList[fakeList.index(i)].count = 1

            #if it is a smali file
            if('.smali' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                with open(dirpath + "\\" + file_name) as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in methods_to_find:
                            if(i in line and 'invoke' in line):

                                #Increase counter for found item
                                myList[fakeList.index(i)].count += 1

In [8]:
#function saves to csv file
def saveToFile(df):
    df.to_csv('benign_csv.csv', mode='a', header=False)  

In [9]:
def appendToDF(list, df):
    #   Initialize empty list for the counts only.
    listOfCount = []
    
    #   Fill such list (except if it's the last one [Name])
    for i in range(len(list)):
        if(i < len(list)-1):
            listOfCount.append(list[i].count)
        else:
            listOfCount.append(list[i])

    #   Append the list to the dataframe
    df.loc[len(df.index)] = listOfCount

In [10]:
#   for the passed directory, do: 
#   Call find function, append results of 'myList' to the df, save.
def runAnalysis(filename, single_file):
    
    findFunctionsInDir(direct=filename, verbose=False, filename = single_file)
    appendToDF(myList, df)

    #TODO not sure why only this method works, but not backing up the list for efficiency.
    myList.clear()
    for k in perms_to_find:
        myList.append(obj(k))

    for k in methods_to_find:
        myList.append(obj(k))
        
    myList.append('')

In [11]:
#decompile all of the APKs in folder (hardcoded)
# 741 free - before
# 542 free - during
# 494 free - after = 247GB
#decompileAll()

In [12]:
#This line will clear the DF (call if needed)
df = df.iloc[0:0]

direc = "C:\\Users\\Vin\\Desktop\\python-ransomware-analyzer\\decompiled_benign"
lim_counter = 0
#   Limits the number of APKs
LIMIT = 2300

#   Here, filename means folder
#   The tqdm is the loading bar library
for filename in tqdm(os.listdir(direc)):

    if(lim_counter < LIMIT):
        try:
            #   If this line does not work, check to see if path is correct
            runAnalysis(os.path.join(direc, filename), filename[4:])

            #   Increase lim_counter
            lim_counter += 1
        except UnicodeDecodeError:
            print("Unicode Error on file: " + filename[4:])
        except FileNotFoundError:
            print("File Not Found on file: " + filename[4:])

saveToFile(df)
print("..done! Saved to csv.")

  0%|          | 2/588 [00:59<4:19:16, 26.55s/it]

Unicode Error on file: com.banca.slot


  1%|          | 3/588 [01:15<3:35:16, 22.08s/it]

Unicode Error on file: com.bandagames.mpuzzle.gp


  2%|▏         | 9/588 [05:43<10:07:11, 62.92s/it]

...

Unicode Error on file: com.creative.Learn.to.draw.comics


100%|█████████▉| 587/588 [7:16:27<00:57, 57.98s/it]

File Not Found on file: com.creative.Learn.to.draw.flowers


100%|██████████| 588/588 [7:17:12<00:00, 44.61s/it]

Unicode Error on file: com.creative.Learn.to.draw.princess
..done! Saved to csv.


In [13]:
saveToFile(df)
print("..done! Saved to csv.")

..done! Saved to csv.


In [14]:
df

,READ_CALENDAR,WRITE_CALENDAR,CAMERA,READ_CONTACTS,WRITE_CONTACTS,GET_ACCOUNTS,ACCESS_FINE_LOCATION,ACCESS_COARSE_LOCATION,RECORD_AUDIO,READ_PHONE_STATE,...,shareIntent,readPhoneNumber,FileInputStream,getSharedPreferences,telephony,getMessageBody,getDisplayOriginatingAddress,wallpaper,crypto,Name
0,0,0,0,0,0,0,0,0,0,1,...,0,0,67,68,29,0,0,0,74,com.banca
1,0,0,1,0,0,0,0,0,0,1,...,0,0,60,17,3,0,0,0,39,com.bandai.zakstorm
2,0,0,0,0,0,0,0,0,0,0,...,0,0,66,49,27,0,0,0,87,com.bandainamcoent.narutoblazingna
3,0,0,0,0,0,0,0,0,0,0,...,0,0,42,12,5,0,0,0,24,com.bandainamcoent.ninjavoltage_app
4,0,0,0,0,0,0,0,0,0,0,...,0,0,78,75,5,0,0,0,83,com.bandainamcogames.dbzdokkanww
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380,0,0,0,0,0,0,0,0,0,1,...,0,0,30,19,3,0,0,0,24,com.CrazyDiamond.LoveBeatMobile_Global
381,0,0,0,1,0,0,0,0,0,0,...,0,0,118,50,12,0,0,0,24,com.crazygame.inputmethod.keyboard7
382,1,1,1,0,0,1,0,0,1,1,...,0,0,54,6,13,0,0,0,79,com.crazylabs.hotel.transylvania.adventure
383,0,0,1,1,0,0,1,0,0,0,...,0,0,233,136,108,0,0,0,218,com.crazylabs.light.it.up


In [15]:
#This line will clear the DF (call if needed)
#df = df.iloc[0:0]

#TODO: Add a way to stop/resume analysis